You’re given JSON:

[
  {"user_id": "123", "email": "TEST@MAIL.com", "signup_date": "2022/01/02"},
  {"user_id": "abc", "email": "invalid.com", "signup_date": "2021-05-03"}
]


👉 Build a Pydantic model that:

- Casts user_id to int (or errors).
- Validates email format.
- Converts signup_date to datetime.
- Skips invalid records gracefully.

In [1]:
from pydantic import BaseModel, field_validator, EmailStr
from datetime import datetime
import json
import logging

logging.basicConfig(
    level=logging.WARNING,
    format="%(levelname)s: %(message)s",
    filename="records.log",
    filemode="w"
)

class Record(BaseModel):
    user_id: int
    signup_date: datetime
    email: EmailStr

    @field_validator('signup_date', mode='before')
    def parse_date(cls, v):
        # Try multiple formats
        for fmt in ("%Y-%m-%d", "%Y/%m/%d"):
            try:
                return datetime.strptime(v, fmt)
            except ValueError:
                continue
        raise ValueError(f"Invalid date format: {v}")


class DataParser:
    def __init__(self, str_data: str):
        self.raw_data = json.loads(str_data)
        self.records: list[Record] = []

    def parse(self):
        for entry in self.raw_data:
            try:
                self.records.append(Record(**entry))
            except Exception as e:
                logging.warning(f"Skipping bad record {entry}: {e}")

    def print_data(self):
        for r in self.records:
            print(r.model_dump())   # model_dump replaces dict() in Pydantic v2


# Example JSON input
data = '''
[
  {"user_id": "123", "email": "TEST@MAIL.com", "signup_date": "2022/01/02"},
  {"user_id": "abc", "email": "invalid.com", "signup_date": "2021-05-03"}
]
'''

parser = DataParser(data)
parser.parse()
parser.print_data()


{'user_id': 123, 'signup_date': datetime.datetime(2022, 1, 2, 0, 0), 'email': 'TEST@mail.com'}
